# Tarea primer día. Bases de datos.
Haremos una base de datos sencilla para almacenar artículos científicos, sus autores y las universidades de estos.

Primero obtenemos los objetos necesarios de las librearias para crear nuestra base de datos

In [1]:
from sqlalchemy import create_engine

motor = create_engine('sqlite:///Articulos.db')
conex = motor.connect()

from sqlalchemy.orm import sessionmaker

session = sessionmaker(bind=motor)()

Ahora creamos 3 tablas. Una que contendra los articulos, otra que contendra los autores y otra que contendra las universidades de origen.

In [2]:
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy_explore
from sqlalchemy import Column, Date, ForeignKey, Integer, NVARCHAR, Numeric, Sequence
from sqlalchemy.orm import relationship

classBase = declarative_base(cls=sqlalchemy_explore.ReflectiveMixin) # base class including utils like an __repr__ method

class Articulos(classBase):
    __tablename__ = 'articulos'
    
    ArtId = Column(Integer, Sequence('articulo_id_seq'), primary_key=True)
    Title = Column(NVARCHAR(150), nullable=False)
    Journal = Column(NVARCHAR(100), nullable=False)
    Year = Column(Date)
    
class Universidades(classBase):
    __tablename__ = 'universidades'
    
    UniId = Column(Integer, Sequence('universidad_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(50), nullable=False)
    Country = Column(NVARCHAR(50), nullable=False)
    
class Autores(classBase):
    __tablename__ = 'autores'
    
    AuthId = Column(Integer, Sequence('autor_id_seq'), primary_key=True)
    FirstName = Column(NVARCHAR(50), nullable=False)
    LastName = Column(NVARCHAR(50), nullable=False)
    ArtId = Column(ForeignKey('articulos.ArtId'), nullable=False, index=True)
    UniId = Column(ForeignKey('universidades.UniId'), nullable=False, index=True)
    
    articulos = relationship('Articulos')
    universidades = relationship('Universidades')
    

classBase.metadata.create_all(motor)

Ahora podemos añadir los datos. Primero añadimos los artículos.

In [3]:
import datetime

art1 = Articulos(
    ArtId = 1,
    Title = 'Sobre el envejecimiento de las bellotas',
    Journal = 'Revista de la Belloteria',
    Year = datetime.date(year=1997, month=7, day=15)
)
art2 = Articulos(
    ArtId = 2,
    Title = 'Sobre la autoridad',
    Journal = 'Monthly Review',
    Year = datetime.date(year=1872, month=4, day=12)
)
art3 = Articulos(
    ArtId = 3,
    Title = 'Por que socialismo',
    Journal = 'Monthly Review',
    Year = datetime.date(year=1949, month=2, day=27)
)

session.add(art1)
session.add(art2)
session.add(art3)

Ahora las autoras.

In [4]:
Engels = Autores(
    AuthId = 1,
    FirstName = 'Friedrich',
    LastName = 'Engels',
    ArtId = 2,
    UniId = 3
)

Martina = Autores(
    AuthId = 2,
    FirstName = 'Martina',
    LastName = 'Rodriguez',
    ArtId = 1,
    UniId = 1
)

Einstein = Autores(
    AuthId = 3,
    FirstName = 'Albert',
    LastName = 'Einstein',
    ArtId = 3,
    UniId = 3
)

Laura = Autores(
    AuthId = 4,
    FirstName = 'Laura',
    LastName = 'Amoedo',
    ArtId = 1,
    UniId = 2
)

session.add(Engels)
session.add(Einstein)
session.add(Martina)
session.add(Laura)

Por último las universidades.

In [5]:
Barcelona = Universidades(
    UniId = 1,
    Name = 'Universidad de Barcelona',
    Country = 'Spain, Catalunya'
)

Pontevedra = Universidades(
    UniId = 2,
    Name = 'Universidad de Pontevedra',
    Country = 'Spain, Galicia'
)

Berlin = Universidades(
    UniId = 3,
    Name = 'Universidad de Berlin',
    Country = 'Berlin'
)


session.add(Barcelona)
session.add(Pontevedra)
session.add(Berlin)

Guardamos (*Commiteamos*) los cambios que hemos hecho en la base de datos.

In [6]:
session.commit()

Ya podemos realizar las consultas. Primero realizamos una para encontrar las universidades de origen de los autores.

In [7]:
from sqlalchemy.sql import text
s = text(
     "SELECT autores.FirstName ||' '|| autores.LastName , universidades.Name "
         "FROM autores,  universidades "
         "WHERE autores.UniId = universidades.UniId")

conex.execute(s).fetchall()


[('Friedrich Engels', 'Universidad de Berlin'),
 ('Martina Rodriguez', 'Universidad de Barcelona'),
 ('Albert Einstein', 'Universidad de Berlin'),
 ('Laura Amoedo', 'Universidad de Pontevedra')]

Ahora realizamos otro ejemplo donde encontramos los paises y el año de los articulos.

In [8]:
t = text(
    "SELECT articulos.Title, universidades.Country, articulos.year  "
        "FROM articulos, autores, universidades "
        "WHERE autores.UniId = universidades.UniId "
        "AND articulos.ArtId = autores.ArtId "
        "ORDER BY articulos.Title "
)

conex.execute(t).fetchall()

# 

[('Por que socialismo', 'Berlin', '1949-02-27'),
 ('Sobre el envejecimiento de las bellotas', 'Spain, Catalunya', '1997-07-15'),
 ('Sobre el envejecimiento de las bellotas', 'Spain, Galicia', '1997-07-15'),
 ('Sobre la autoridad', 'Berlin', '1872-04-12')]